In [2]:
%pip install -U langchain

  Using cached langgraph_sdk-0.3.3-py3-none-any.whl.metadata (1.6 kB)
Using cached langgraph_sdk-0.3.3-py3-none-any.whl (67 kB)

  Attempting uninstall: langgraph-sdk

    Found existing installation: langgraph-sdk 0.2.9

    Uninstalling langgraph-sdk-0.2.9:

      Successfully uninstalled langgraph-sdk-0.2.9

  Attempting uninstall: langgraph-prebuilt

    Found existing installation: langgraph-prebuilt 1.0.1

    Uninstalling langgraph-prebuilt-1.0.1:

      Successfully uninstalled langgraph-prebuilt-1.0.1

  Attempting uninstall: langgraph

    Found existing installation: langgraph 1.0.1

   -------------------- ------------------- 2/4 [langgraph]
    Uninstalling langgraph-1.0.1:
   -------------------- ------------------- 2/4 [langgraph]
      Successfully uninstalled langgraph-1.0.1
   -------------------- ------------------- 2/4 [langgraph]
   -------------------- ------------------- 2/4 [langgraph]
   -------------------- ------------------- 2/4 [langgraph]
  Attempting unin